## 训练与预测

### 导入模块

In [1]:
# 不变的模块
import os
from Utilties.setup import import_torch
import_torch() # 导入torch位置到sys环境变量中
import torch
from torch import optim
from torch.utils.data import DataLoader, sampler, random_split
import torch.nn.functional as F
from torchvision import datasets,transforms as T

import importlib # 用于导入自定义模块

d:\XXXDevelopment\ML-DL\ML_tj\TJML_ENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 这些是自行实现的，可能动态改变的模块
# 首次导入
from Utilties.my_tools import get_classes
from Utilties.my_preprocess_nn import MyDataset

from Classifiers.my_networks import MyResNet18,ResBlock
# 重新导入可能变化的模块
importlib.reload(importlib.import_module("Utilties.my_tools"))
importlib.reload(importlib.import_module("Utilties.my_preprocess_nn"))
importlib.reload(importlib.import_module("Classifiers.my_networks"))

<module 'Classifiers.my_networks' from 'd:\\XXXDevelopment\\ML-DL\\ML_tj\\Lab2_Classification\\Classifiers\\my_networks.py'>

### 加载数据与预处理


In [3]:
dataset_path = "D:\\TJCS\\ML&DL\\datasets\\butterfly_moths"
trainset_path = "D:\\TJCS\\ML&DL\\datasets\\butterfly_moths\\train"
testset_path = "D:\\TJCS\\ML&DL\\datasets\\butterfly_moths\\test"
valset_path = "D:\\TJCS\\ML&DL\\datasets\\butterfly_moths\\valid"

# 获取数据集的类别
classes = get_classes(trainset_path)
print(classes, len(classes))




['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO', 'ARCIGERA FLOWER MOTH', 'ATALA', 'ATLAS MOTH', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BANDED TIGER MOTH', 'BECKERS WHITE', 'BIRD CHERRY ERMINE MOTH', 'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROOKES BIRDWING', 'BROWN ARGUS', 'BROWN SIPROETA', 'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHALK HILL BLUE', 'CHECQUERED SKIPPER', 'CHESTNUT', 'CINNABAR MOTH', 'CLEARWING MOTH', 'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMET MOTH', 'COMMON BANDED AWL', 'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH', 'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE', 'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'EMPEROR GUM MOTH', 'GARDEN TIGER MOTH', 'GIANT LEOPARD MOTH', 'GLITTERING SAPPHIRE', 'GOLD BANDED', 'GREAT EGGFLY', 'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREEN HAIRSTREAK', 'GREY HAIRSTREAK', 'HERCULES MOTH', 'HUMMING BIRD HAWK MOTH', 'INDRA SWALLOW', 'IO MOTH', 'Iphiclus si

In [4]:
transform_train_list = [
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
    # T.RandomErasing(p=0.2, value='random')
]

transform_test_list = [
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
]

In [5]:
# 加载数据

bm_data = MyDataset(
    dataset_path,
    transform_train_list=transform_train_list,
    transform_test_list=transform_test_list,
)

print(bm_data)




MyDataset metadata dir:  D:\TJCS\ML&DL\datasets\butterfly_moths
MyDataset data length:  13594
MyDataset data mean:  tensor([0.4845, 0.4676, 0.3441])
MyDataset data std:  tensor([0.2334, 0.2277, 0.2219])
MyDataset


In [6]:
batch_size = 2

train_loader = DataLoader(
    bm_data.train_data,
    batch_size=batch_size,
    shuffle=True,
    #   sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))
)
test_loader = DataLoader(
    bm_data.test_data,
    batch_size=batch_size,
    #   sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_TRAIN + NUM_VAL))
    shuffle=True,
)

### 导入模型训练

#### 定义训练和优化器

In [ ]:
# 定义各种参数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32 
display_itv = 1000

# 模型
model =MyResNet18(ResBlock,[3,3,3,3],64,100)  #MyModel()
model_name='resnet3333'
# 超参数
learning_rate = 4e-3
# epoch
train_epochs=20

model_fname = f'{model_name}_bs{batch_size}_ep{train_epochs}_lr{learning_rate}'
best_model_path="./model"+ model_fname 
best_model=None
best_val_acc=0

In [ ]:
# 准确率
def check_accuracy(loader, model):   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        
    return acc

In [7]:


def train(model, optimizer, epochs=1):
    """
    Train a model 
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            # print(x.shape)
            # summary(model,x.shape)
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % display_itv == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=train_epochs)

In [ ]:
print(best_val_acc)
torch.save(best_model.state_dict(), best_model_path+f'_acc{best_val_acc}'+'.pth')       
check_accuracy(test_loader, best_model)

# 原始Resnet18  bs=32  ep=10 lr=1e-3 :Got 413 / 500 correct (82.60)
# REsnet-? [3,3,3,3] bs=64  ep=10 lr=3e-3: Got 418 / 500 correct (83.60)

## 分析与可视化

In [13]:
import netron
import torch.onnx

modelData = "D:\TJCS\ML&DL\ML\Lab2\model\\myResnet18_bs64_ep20_lr0.004_acc0.888_script.onnx"
netron.start(modelData)  # 输出网络结构


Serving 'D:\TJCS\ML&DL\ML\Lab2\model\myResnet18_bs64_ep20_lr0.004_acc0.888_script.onnx' at http://localhost:8080


('localhost', 8080)